In [9]:
import tiktoken
import re
import pandas as pd
prompt= '''For each drug trial group described below, identify the maximum dosage mentioned. If a maximum dosage is not specified, use the provided standard dose. Convert the dosage to a daily total in milligrams per day (mg/day), considering the number of doses per day. Note the units of slices, times, hours, etc.
Format your response as follows: "The dosage is xxx mg/day." If information about a drug or its dosage is missing, respond with "None".
'''
prompt = '''Analyze each drug trial group described. For each group:
1. Identify the maximum dosage mentioned.
2. If no maximum dosage is specified, record the standard dose.
3. Convert all dosages to a daily total in milligrams per day (mg/day), factoring in the frequency of doses per day.
4. Ensure all units (e.g., pills, times, hours) are considered in the conversion.

Format your response as follows:
- "The dosage is xxx mg/day."
- If information is missing, respond with "None".
-No explanations.

Description:
'''

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
token_num = encoding.encode(prompt)
print("Token数量:",len(token_num))
# token_num = encoding.encode(result[0])
# print("Token数量：",len(token_num))

Token数量: 115


In [17]:
import openai
from packaging import version
current_version = version.parse(openai.__version__)
print(current_version)
from openai import OpenAI
client = OpenAI(api_key='')

1.2.0


In [2]:
descrip = pd.read_csv('dose_description.csv', index_col=0)['dose_description'].apply(lambda x: x if 'mg' in x else None).dropna()

In [3]:
def dropm2(x):
    if pd.isnull(x):
        return None
    if 'm2' in x:
        return None
    if 'm^2' in x:
        return None
    if 'm²' in x:
        return None
    return x
descrip = pd.DataFrame(descrip)
descrip['dose_description'] = descrip['dose_description'].apply(dropm2)
descrip = descrip.dropna()
print(len(descrip))

13320


In [4]:
descrip = pd.DataFrame(descrip)
descrip.insert(1, 'mg/kg to mg', descrip['dose_description'])
descrip

,dose_description,mg/kg to mg
nctid,,
NCT00000271,"['Participants were treated with desipramine, ...","['Participants were treated with desipramine, ..."
NCT00000304,"['15/30 mg d-amphetamine', '30/60 mg d-ampheta...","['15/30 mg d-amphetamine', '30/60 mg d-ampheta..."
NCT00000308,['15 mg of d-amphetamine for first 8 weeks of ...,['15 mg of d-amphetamine for first 8 weeks of ...
NCT00000317,"['Placebo plus relapse prevention counseling',...","['Placebo plus relapse prevention counseling',..."
NCT00000392,['Peptide T given intranasally at a dosage of ...,['Peptide T given intranasally at a dosage of ...
...,...,...
NCT06260072,['400 mg of Magnesium Oxide and 400 mg Ribofla...,['400 mg of Magnesium Oxide and 400 mg Ribofla...
NCT06260891,['25 mg of zinc as zinc gluconate taken orally...,['25 mg of zinc as zinc gluconate taken orally...
NCT06261125,['This arm includes the patients who previousl...,['This arm includes the patients who previousl...


In [5]:
def transform_mgkg_to_mg(text):
    weight_kg = 60
    dosages = re.findall(r"\d+\.?\d*\s*mg/kg", text)
    if len(dosages) == 0:
        return text
    dosage_values = [float(re.findall(r"\d+\.?\d*", dosage)[0]) for dosage in dosages]
    calculated_dosages = [int(dosage * weight_kg) for dosage in dosage_values]
    for original, calculated in zip(dosages, calculated_dosages):
        text = text.replace(original, f"{calculated} mg")
    return text

def transform_mgkghour_to_mg(text):
    text = text.replace('/hour', '/hr')
    dosages = re.findall(r"\d+\.?\d*\s*mg/hr", text)
    if len(dosages) == 0:
        return text
    dosage_values = [float(re.findall(r"\d+\.?\d*", dosage)[0]) for dosage in dosages]
    calculated_dosages = [int(dosage * 24) for dosage in dosage_values]
    for original, calculated in zip(dosages, calculated_dosages):
        print(original, calculated)
        text = text.replace(original, f"{calculated} mg/day")
    return text

for nctid in descrip.index:
    i = descrip['dose_description'][nctid]
    text = '\n'.join(eval(i)) if isinstance(eval(i), list) else i
    text = transform_mgkg_to_mg(text)
    # print(text)
    # text = transform_mgkghour_to_mg(text)
    text = text.replace('mg/d ', 'mg/day ')
    text = text.replace('mg/d,', 'mg/day,')
    text = text.replace('/hour', '/hr')
    descrip.loc[nctid, 'mg/kg to mg'] = text

descrip.to_csv('dose_description_dropm2.csv')

In [13]:
descrip = pd.read_csv('dose_description_dropm2.csv', index_col=0)['mg/kg to mg']
# print(len(descrip))
print(len(descrip))
#= [''.join(i.strip().split()) for i in descrip]
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') #'gpt-3.5-turbo'
total = 0
for i in descrip:
    # print(i)
    token_num = encoding.encode(prompt+i)
    # print(i, "   有 ", len(token_num), " 个tokens")
    total += len(token_num)
print(f"Total tokens: {total}, Price: {total/1000000*2} USD")

13320
Total tokens: 2839650, Price: 5.6793 USD


In [ ]:
client = OpenAI(api_key='')
completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=
            [
                #{"role": "system", "content": "You are a dose conversion assistant, distill the dose and express it as mg/1kg/1day.The unit required directly uses the national standard."},
                {"role": "user", "content": '\n'.join(eval(descrip.loc['NCT00394433']))},
            ]
)
print(completion.choices[0].message.content)

- Bevacizumab: 10 mg/kg IV on day 1 every 3 weeks
- Docetaxel: 30 mg/m2 IV over 30 minutes on days 1 and 8 of each 3-week cycle
- Cisplatin: 25 mg/m2 IV over 30 minutes on days 1 and 8 of each 3-week cycle
- Irinotecan: 50 mg/m2 IV over 30 minutes on days 1 and 8 of each 3-week cycle

Using the national standard unit of mg/1kg/1day, the doses can be converted as follows:
- Bevacizumab: 10 mg/kg/1day
- Docetaxel: 30 mg/m2/1day
- Cisplatin: 25 mg/m2/1day
- Irinotecan: 50 mg/m2/1day


In [21]:
descrip.index

Index(['NCT00000271', 'NCT00000304', 'NCT00000308', 'NCT00000317',
       'NCT00000392', 'NCT00000395', 'NCT00000421', 'NCT00000435',
       'NCT00000457', 'NCT00000588',
       ...
       'NCT06254690', 'NCT06255977', 'NCT06256432', 'NCT06256926',
       'NCT06259929', 'NCT06260072', 'NCT06260891', 'NCT06261125',
       'NCT06263543', 'NCT06264310'],
      dtype='object', name='nctid', length=13320)

In [23]:
for i in range(len(descrip)):
    text = descrip[i]
    print(text)
    if i > 5:
        break

Participants were treated with desipramine, up to 300 mg per day. All patients received weekly individual manual-guided relapse prevention therapy.
Participants were treated with matching placebo. All patients received weekly individual manual-guided relapse prevention therapy.
15/30 mg d-amphetamine
30/60 mg d-amphetamine
placebo
15 mg of d-amphetamine for first 8 weeks of study and 30 mg for the second 8 weeks
30 mg of d-amphetamine for first 8 weeks of study and 60 mg for the second 8 weeks
placebo
Placebo plus relapse prevention counseling
Risperidone (4mg/day) plus relapse prevention counseling
Peptide T given intranasally at a dosage of 2mg 3 times a day for 6 months
Placebo given intranasally at a dosage of 2mg 3 times a day for 6 months
Subjects receiving Methotrexate for 6 weeks and 5 mg of Folinic acid daily for 1 week.
Subjects receiving Methotrexate for 6 weeks and 5 mg of Folic acid daily for 1 week.
Patients who remained clinically stable but showed serologic evidence of 

In [43]:
# labels = []
# num = 0
for i in range(13153, len(descrip)):
    text = descrip[i]
    # if num > 5:
    #     break
    # i = '\n'.join(eval(i)) if isinstance(eval(i), list) else i
    # print(i)
    client = OpenAI(api_key='')
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=
            [
                # {"role": "system", "content": "You are a dose conversion assistant, distill the dose and express it as mg/1kg/1day.The unit required directly uses the national standard."},
                {"role": "user", "content": prompt+text},
            ],
        max_tokens=1000,
    )

    result = completion.choices[0].message.content
    print(result)
    labels.append(result)

label = pd.DataFrame({'nctid': descrip.index, 'GPT':labels})
label.to_csv('dose_label.csv', index=False)

1. The maximum dosage mentioned is 20mg.
2. The dosage is 20 mg/day.
3. None.
- TXA127 group:
1. The maximum dosage mentioned is 0.300 mg.
2. The dosage is 0.300 mg/day.
- Placebo group:
None
- The dosage is 100 mg/day.
- The dosage is 100 mg/day (during the first medication period).
- The dosage is 200 mg/day (during the second medication period).
- The dosage is 300 mg/day.
- The dosage is 15 mg/day.
- The dosage is None.
- ENA-001:
1. The maximum dosage is not specified.
- Placebo:
1. The maximum dosage is not specified.
- "The dosage is 8 mg/day for body weight < 60 kg or 12 mg/day for body weight ≥ 60 kg."
- "The dosage is 360 mg/every 2 weeks, equivalent to 180 mg/day."
- None
- The dosage is 12 mg/day for body weight 60 kg or greater.
- The dosage is 8 mg/day for body weight less than 60 kg.
- The dosage is 75 mg/day.
- The dosage is 125 mg/day.
- The dosage is 50 mg/day.
- The dosage is 100 mg/day.
- None.
- "The dosage is 4.8 mg/day."
- The dosage is 375 mg/day.
- The dosage i

In [42]:
import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (100000,-1))
label = pd.DataFrame({'nctid': descrip.index[:13153], 'GPT':labels}).set_index('nctid')
label.to_csv('dose_label13153.csv')

In [46]:
dose = pd.read_csv('dose_description_dropm2.csv', index_col=0)
dose

,dose_description,mg/kg to mg
nctid,,
NCT00000271,"['Participants were treated with desipramine, ...","Participants were treated with desipramine, up..."
NCT00000304,"['15/30 mg d-amphetamine', '30/60 mg d-ampheta...",15/30 mg d-amphetamine\n30/60 mg d-amphetamine...
NCT00000308,['15 mg of d-amphetamine for first 8 weeks of ...,15 mg of d-amphetamine for first 8 weeks of st...
NCT00000317,"['Placebo plus relapse prevention counseling',...",Placebo plus relapse prevention counseling\nRi...
NCT00000392,['Peptide T given intranasally at a dosage of ...,Peptide T given intranasally at a dosage of 2m...
...,...,...
NCT06260072,['400 mg of Magnesium Oxide and 400 mg Ribofla...,400 mg of Magnesium Oxide and 400 mg Riboflavi...
NCT06260891,['25 mg of zinc as zinc gluconate taken orally...,25 mg of zinc as zinc gluconate taken orally o...
NCT06261125,['This arm includes the patients who previousl...,This arm includes the patients who previously ...


In [51]:
label.set_index('nctid', inplace=True)
dose['GPT'] = label['GPT']

In [53]:
def get_num(x):
    matchs = re.findall(r"(\d+\.?\d*)\s*mg/day",x)
    
    if len(matchs) >= 1:
        matchs = [float(i) for i in matchs]
        return matchs
    return matchs

def mg_select(value):
    if type(value) == str:
        if 'mg/day' in value:
            # print(get_num(value))
            return get_num(value)
        else:
            return None
    elif type(value) == list:
        lis = []
        for i in value:
            if 'mg/day' in i:
                lis.extend(get_num(i))
        if len(lis) > 0:
            # print(lis)
            return lis
        else:
            return None
    return None

dose['values'] = dose['GPT'].apply(mg_select)
dose

,dose_description,mg/kg to mg,GPT,values
nctid,,,,
NCT00000271,"['Participants were treated with desipramine, ...","Participants were treated with desipramine, up...",- The dosage is 300 mg/day.\n- None,[300.0]
NCT00000304,"['15/30 mg d-amphetamine', '30/60 mg d-ampheta...",15/30 mg d-amphetamine\n30/60 mg d-amphetamine...,- The dosage is 75 mg/day.\n- The dosage is 12...,"[75.0, 120.0]"
NCT00000308,['15 mg of d-amphetamine for first 8 weeks of ...,15 mg of d-amphetamine for first 8 weeks of st...,1. The maximum dosage mentioned is 30 mg.\n- T...,"[30.0, 60.0]"
NCT00000317,"['Placebo plus relapse prevention counseling',...",Placebo plus relapse prevention counseling\nRi...,- Placebo plus relapse prevention counseling:\...,"[0.0, 4.0, 4.0, 4.0, 4.0]"
NCT00000392,['Peptide T given intranasally at a dosage of ...,Peptide T given intranasally at a dosage of 2m...,- Peptide T group: The dosage is 6 mg/day.\n- ...,"[6.0, 6.0]"
...,...,...,...,...
NCT06260072,['400 mg of Magnesium Oxide and 400 mg Ribofla...,400 mg of Magnesium Oxide and 400 mg Riboflavi...,"- ""The dosage is 800 mg/day.""\n- ""None""",[800.0]
NCT06260891,['25 mg of zinc as zinc gluconate taken orally...,25 mg of zinc as zinc gluconate taken orally o...,"- ""The dosage is 40 mg/day.""",[40.0]
NCT06261125,['This arm includes the patients who previousl...,This arm includes the patients who previously ...,- The dosage is 12 mg/day.\n- The dosage is 12...,"[12.0, 12.0]"


In [56]:
dose['values_max'] = dose['values'].apply(lambda x: max(x) if x else None)
dose['values_min'] = dose['values'].apply(lambda x: min(x) if x else None)
dose['values_mean'] = dose['values'].apply(lambda x: sum(x)/len(x) if x else None)
dose['values_interval'] = dose['values'].apply(lambda x: max(x)-min(x) if x else None)
dose

In [92]:
def get_median(x):
    n = len(x)
    if n % 2 == 0:
        return (x[n//2-1]+x[n//2])/2
    else:
        return x[n//2]

In [95]:
dose['values_median'] = dose['values'].apply(lambda x: get_median(x) if x else None)

In [59]:
import math

In [81]:
(dose['values_max'] == 0).sum()

25

In [64]:
# 把values_max取log然后去整数
dose['values_max'].apply(lambda x: round(math.log(x, 10)) if x > 0 else None).value_counts()

values_max
 2.0    5148
 3.0    4501
 1.0    2190
 0.0     598
 4.0     277
-1.0      44
 5.0       5
-2.0       2
Name: count, dtype: int64

In [65]:
# 把values_min取log然后去整数
dose['values_min'].apply(lambda x: round(math.log(x, 10)) if x > 0 else None).value_counts()

values_min
 2.0    5179
 3.0    3190
 1.0    2972
 0.0    1026
-1.0     150
 4.0     143
-2.0      10
 5.0       4
-3.0       1
Name: count, dtype: int64

In [101]:
dose['label'] = dose['values_median'].apply(lambda x: math.log(x, 10) if x > 0 else None)

In [103]:
pd.DataFrame(dose['label']).to_csv('dose_target.csv')

In [72]:
temp = dose['values'][~(dose['values_min'].apply(lambda x: round(math.log(x, 10)) if x > 0 else None) == dose['values_max'].apply(lambda x: round(math.log(x, 10)) if x > 0 else None))]

In [83]:
def get_log_interval(x):
    if x:
        maxin = max(x)
        minin = min(x)
        return [round(math.log(maxin, 10)) if maxin!=0 else 0, round(math.log(minin, 10)) if minin!=0 else 0]
    
    return None

temp.apply(get_log_interval).value_counts()

values
[3, 2]      1019
[2, 1]       816
[1, 0]       275
[3, 1]       263
[2, 0]       191
[3, 0]       112
[4, 3]        93
[0, -1]       60
[0, 0]        31
[4, 2]        31
[1, -1]       30
[2, -1]       11
[3, -1]       10
[4, 1]         9
[-1, -2]       5
[0, -2]        2
[-2, -3]       1
[4, 0]         1
[1, -2]        1
[5, 0]         1
[2, -2]        1
Name: count, dtype: int64

In [66]:
dose['values_interval'].apply(lambda x: round(math.log(x, 10)) if x > 0 else None).value_counts()

values_interval
 2.0    2070
 3.0    1608
 1.0     923
 0.0     247
 4.0      89
-1.0      35
 5.0       3
-2.0       2
Name: count, dtype: int64

In [141]:
def transform_mgkg_to_mg(text):
    weight_kg = 60
    dosages = re.findall(r"\d+\.?\d*\s*mg/kg", text)
    if len(dosages) == 0:
        return text
    dosage_values = [float(re.findall(r"\d+\.?\d*", dosage)[0]) for dosage in dosages]
    calculated_dosages = [int(dosage * weight_kg) for dosage in dosage_values]
    for original, calculated in zip(dosages, calculated_dosages):
        text = text.replace(original, f"{calculated} mg")
    return text

def transform_mgkghour_to_mg(text):
    text = text.replace('/hour', '/hr')
    dosages = re.findall(r"\d+\.?\d*\s*mg/hr", text)
    if len(dosages) == 0:
        return text
    dosage_values = [float(re.findall(r"\d+\.?\d*", dosage)[0]) for dosage in dosages]
    calculated_dosages = [int(dosage * 24) for dosage in dosage_values]
    for original, calculated in zip(dosages, calculated_dosages):
        print(original, calculated)
        text = text.replace(original, f"{calculated} mg/day")
    return text

for nctid in descrip.index:
    i = descrip['dose_description'][nctid]
    text = '\n'.join(eval(i)) if isinstance(eval(i), list) else i
    text = transform_mgkg_to_mg(text)
    # print(text)
    # text = transform_mgkghour_to_mg(text)
    text = text.replace('mg/d ', 'mg/day ')
    text = text.replace('mg/d,', 'mg/day,')
    text = text.replace('/hour', '/hr')
    descrip.loc[nctid, 'mg/kg to mg'] = text
